# Exercise 2
Today we are going to continue work on pointclouds.
We will work on trying to cluster pointclouds to be able to segment them.
    
If you do not have sklearn installed make sure to **pip install scikit-learn**

In [1]:
import numpy as np
import open3d as o3d
import copy
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.cluster import KMeans, k_means

In [2]:
def draw_labels_on_model(pcl,labels):
    #给不同的cluster画上不同的颜色
    cmap = plt.get_cmap("tab20")
    pcl_temp = copy.deepcopy(pcl)
    max_label = labels.max()
    print("%s has %d clusters" % (pcl_name, max_label + 1))
    colors = cmap(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    pcl_temp.colors = o3d.utility.Vector3dVector(colors[:, :3])
    o3d.visualization.draw_geometries([pcl_temp])


## On a cube.
We create a point cloud using open3d.
Our goal is to segment each side using k means.

In [3]:
pcl_name = 'Cube'
density = 1e4 # density of sample points to create
pcl = o3d.geometry.TriangleMesh.create_box().sample_points_uniformly(int(density))
eps = 0.4
print("%s has %d points" % (pcl_name, np.asarray(pcl.points).shape[0]))
o3d.visualization.draw_geometries([pcl])

Cube has 10000 points


If we just use Kmeans out of the box with the pointcloud we will get the following


Note that pressing plus and minus in the viewer will increase/decrease the size of the points in the viewer

In [5]:
km = KMeans(n_clusters=6, init='random',
            n_init=10, max_iter=300, tol=1e-04, random_state=0)

# Get the points from the pointcloud as nparray
xyz = np.asarray(pcl.points)
labels = km.fit_predict(xyz)
draw_labels_on_model(pcl, labels)

Cube has 6 clusters


As we can see we get 6 clusters but they do not span a side.

We try again but this time we instead use the normals of the cube.
The normals for each plane should be parallel with the other normals from said plane.

In [6]:
###
# Code goes here
pcl.estimate_normals() # add normals
normals = pcl.normals
normals = np.asarray(normals) # extract normals
labels = km.fit_predict(normals)
draw_labels_on_model(pcl, labels)
###

Cube has 6 clusters


This still does not work, the opposite side will also have normals that point the other way which will parallel.

So to combat this we can attempt to use the xyz coordinates and the normals.

In [7]:
###
# Code goes here
xyz_n = np.concatenate((xyz, normals), axis=1)
labels = km.fit_predict(xyz_n)
draw_labels_on_model(pcl, labels)
## state 1 and 4 are still confusing
###

Cube has 6 clusters


## Exercises

### A) K means continued.

Combine the point cloud points (xyz) with the normals and do k-means.

```xyz_n = np.concatenate((xyz, normals), axis=1)```

Do you get better clusters?
Why would adding the normals help?

### B) 
Try weighting either the points or normals by scaling them by some factor, can segment each of the faces of the cube?
### C)
Try to cluster all the different shapes using k means.
```{python}
d = 4
mesh = o3d.geometry.TriangleMesh.create_tetrahedron().translate((-d, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_octahedron().translate((0, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_icosahedron().translate((d, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_torus().translate((-d, -d, 0))
mesh += o3d.geometry.TriangleMesh.create_moebius(twists=1).translate(
    (0, -d, 0))
mesh += o3d.geometry.TriangleMesh.create_moebius(twists=2).translate(
    (d, -d, 0))
mesh.sample_points_uniformly(int(1e5)), 0.5
```

### D)
Now try with the pointcloud in "pointclouds/fragment.ply"
Are you able to cluster the pointcloud?

What features here would it make sense to cluster?
- fpfh features?
- xyz
- normals 
- colors

Are you able to get clusters that make sense? Why?

### E)
Use the built in cluster_dbscan algorithm.
Tweak the parameters and see what you get out.

Attempt on the combined figures and on "fragment.ply"
```{Python}
#eps (float) – Density parameter that is used to find neighbouring points.
eps = 0.02

#min_points (int) – Minimum number of points to form a cluster.
min_points = 10

labels = np.array(pcl.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))
```


In [8]:
# B
alpha = 0.5
beta = 0.3
xyz_n = np.concatenate((alpha*xyz, beta*normals), axis=1)
labels = km.fit_predict(xyz_n)
draw_labels_on_model(pcl, labels)
# The result is appearantly better

Cube has 6 clusters


In [9]:
# C
d = 4
mesh = o3d.geometry.TriangleMesh.create_tetrahedron().translate((-d, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_octahedron().translate((0, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_icosahedron().translate((d, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_torus().translate((-d, -d, 0))
mesh += o3d.geometry.TriangleMesh.create_moebius(twists=1).translate(
    (0, -d, 0))
mesh += o3d.geometry.TriangleMesh.create_moebius(twists=2).translate(
    (d, -d, 0))
pcl = mesh.sample_points_uniformly(int(1e5))
km = KMeans(n_clusters=6, init='random',
            n_init=10, max_iter=300, tol=1e-04, random_state=0)
xyz = np.asarray(pcl.points)
labels = km.fit_predict(xyz)
draw_labels_on_model(pcl, labels)

Cube has 6 clusters


In [10]:
# D
pcl = o3d.io.read_point_cloud("TestData/fragment.ply")
## extract features
pcl.estimate_normals()
normals = np.asarray(pcl.normals)
xyz = np.asarray(pcl.points)
f = o3d.registration.compute_fpfh_feature(pcl, 
o3d.geometry.KDTreeSearchParamHybrid(radius=0.15, max_nn=100
))
feature = np.array(f.data)
color = np.asarray(pcl.colors)
fusion = np.concatenate((xyz, normals, np.transpose(feature),color), axis=1)
o3d.visualization.draw_geometries([pcl])

In [11]:
### K-means clustering
km = KMeans(n_clusters=10, init='random',
            n_init=10, max_iter=300, tol=1e-04, random_state=0)
labels = km.fit_predict(xyz)
draw_labels_on_model(pcl, labels)

Cube has 10 clusters


In [12]:
labels = km.fit_predict(normals)
draw_labels_on_model(pcl, labels)
# for me, normal is the best feature, since I wanna the meaning of the clusters to be the 6 faces of the object. After done the next question, I revise k to be 10 to seperate the parts inside the object, normal is still the best feature.

Cube has 10 clusters


In [14]:
labels = km.fit_predict(color)
draw_labels_on_model(pcl, labels)

Cube has 10 clusters


In [15]:
labels = km.fit_predict(feature)
draw_labels_on_model(pcl, labels)

Cube has 10 clusters


In [16]:
# E
#eps (float) – Density parameter that is used to find neighbouring points.
eps = 0.02

#min_points (int) – Minimum number of points to form a cluster.
min_points = 10

labels = np.array(pcl.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))
draw_labels_on_model(pcl, labels)


Cube has 10 clusters
